In [1]:
# Import libraries
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from urllib.request import urlopen, Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer





Recent News Headlines for AAPL: 
Dow Jones Futures: Federal Reserve, Tesla 'Confidence' Stem Growth Sell-Off; Twilio, Tilray Lead Earnings Movers ( Feb-17-21 07:32PM )
These Are The Best Robinhood Stocks To Buy Or Watch Now ( 06:00PM )
Warren Buffett Stocks: What's Inside Berkshire Hathaway's Portfolio? ( 05:01PM )


Recent News Headlines for TSLA: 
Dow Jones Futures: Federal Reserve, Tesla 'Confidence' Stem Growth Sell-Off; Twilio, Tilray Lead Earnings Movers ( Feb-17-21 06:59PM )
Elon Musk Loses No. 1 Status As Tesla Stock Falls, But SpaceX Value Jumps ( 04:08PM )
Ford Launches Tesla Rival In Europe Amid Continent-Wide EV Revamp ( 04:07PM )


Recent News Headlines for AMZN: 
RPT-Indian retailer group calls for ban on Amazon in country after Reuters report ( Feb-17-21 06:30PM )
Snowstorms, Power Outages, Frigid Temperatures Impacting Retailers and Consumers in the U.S. ( 05:16PM )
Elon Musk Loses No. 1 Status As Tesla Stock Falls, But SpaceX Value Jumps ( 04:08PM )


LookupError: 
**********************************************************************
  Resource [93mvader_lexicon[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('vader_lexicon')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93msentiment/vader_lexicon.zip/vader_lexicon/vader_lexicon.txt[0m

  Searched in:
    - 'C:\\Users\\lemon/nltk_data'
    - 'C:\\Users\\lemon\\anaconda3\\nltk_data'
    - 'C:\\Users\\lemon\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\lemon\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\lemon\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [ ]:
# Parameters 
n = 3 #the # of article headlines displayed per ticker
tickers = ['AAPL', 'TSLA', 'AMZN']

# Get Data
finwiz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
    resp = urlopen(req)    
    html = BeautifulSoup(resp, features="lxml")
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table

try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')
    
        print ('\n')
        print ('Recent News Headlines for {}: '.format(ticker))
        
        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            print(a_text,'(',td_text,')')
            if i == n-1:
                break
except KeyError:
    pass

# Iterate through the news
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        text = x.a.get_text() 
        date_scrape = x.td.text.split()

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        else:
            date = date_scrape[0]
            time = date_scrape[1]

        ticker = file_name.split('_')[0]
        
        parsed_news.append([ticker, date, time, text])
        
# Sentiment Analysis
analyzer = SentimentIntensityAnalyzer()

columns = ['Ticker', 'Date', 'Time', 'Headline']
news = pd.DataFrame(parsed_news, columns=columns)
scores = news['Headline'].apply(analyzer.polarity_scores).tolist()

df_scores = pd.DataFrame(scores)
news = news.join(df_scores, rsuffix='_right')


# View Data 
news['Date'] = pd.to_datetime(news.Date).dt.date

unique_ticker = news['Ticker'].unique().tolist()
news_dict = {name: news.loc[news['Ticker'] == name] for name in unique_ticker}

values = []
for ticker in tickers: 
    dataframe = news_dict[ticker]
    dataframe = dataframe.set_index('Ticker')
    dataframe = dataframe.drop(columns = ['Headline'])
    print ('\n')
    print (dataframe.head())
    
    mean = round(dataframe['compound'].mean(), 2)
    values.append(mean)
    
df = pd.DataFrame(list(zip(tickers, values)), columns =['Ticker', 'Mean Sentiment']) 
df = df.set_index('Ticker')
df = df.sort_values('Mean Sentiment', ascending=False)
print ('\n')
print (df)